<a href="https://colab.research.google.com/github/ShotaSasaki-HU/ASTRO-CAMP-2025/blob/main/linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ライブラリ

In [136]:
!pip install earthengine-api geemap rasterio folium scikit-learn --quiet
!pip install rasterio
import rasterio
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import ee
import geemap
import geopandas as gpd
from shapely.geometry import box
import datetime
!pip install japanize-matplotlib
import japanize_matplotlib

# GEEの認証

In [137]:
ee.Authenticate() # Colabではブラウザで認証を行う
ee.Initialize(project='astro-camp')
# ee.Initialize(project='astrocamp')

# 被害率（正解データ）のdf_damage_ratio

In [149]:
prefectures = ["福岡", "佐賀", "長崎", "大分", "熊本", "宮崎", "鹿児島"]

# 初期データフレーム
cols = ["都道府県", "year", "被害面積（ha）", "被害量（t）", "被害率（%）"]
df_damage_ratio = pd.DataFrame(columns=cols)

for prefecture in prefectures:
    # 2016
    year = 2016
    path_csv = f"https://github.com/ShotaSasaki-HU/ASTRO-CAMP-2025/raw/refs/heads/main/data/%E6%B0%B4%E7%A8%B2%E3%81%AE%E8%A2%AB%E5%AE%B3%E9%9D%A2%E7%A9%8D%E5%8F%8A%E3%81%B2%E3%82%99%E8%A2%AB%E5%AE%B3%E9%87%8F%EF%BC%88%E5%85%A8%E5%9B%BD%E8%BE%B2%E6%A5%AD%E5%9C%B0%E5%9F%9F%E5%88%A5%E3%83%BB%E9%83%BD%E9%81%93%E5%BA%9C%E7%9C%8C%E5%88%A5%EF%BC%89/{year}_%E6%B0%B4%E7%A8%B2%E3%81%AE%E8%A2%AB%E5%AE%B3%E9%9D%A2%E7%A9%8D%E5%8F%8A%E3%81%B2%E3%82%99%E8%A2%AB%E5%AE%B3%E9%87%8F%EF%BC%88%E5%85%A8%E5%9B%BD%E8%BE%B2%E6%A5%AD%E5%9C%B0%E5%9F%9F%E5%88%A5%E3%83%BB%E9%83%BD%E9%81%93%E5%BA%9C%E7%9C%8C%E5%88%A5%EF%BC%89.csv"
    df = pd.read_csv(path_csv, encoding="shift-jis", header=0)

    target_col = [col for col in df.columns if "全国農業地域" in col][0] # 「全国農業地域」を含む列名を抽出
    df = df[df[target_col].str.contains(prefecture)]

    target_col = [col for col in df.columns if "項目" in col][0]
    df_damaged_area = df[df[target_col] == "虫害_ウンカ_被害面積"]
    df_damaged_amount = df[df[target_col] == "虫害_ウンカ_被害量"]
    df_damaged_ratio = df[df[target_col] == "虫害_ウンカ_被害率"]

    df_damage_ratio.loc[len(df_damage_ratio)] = [
        prefecture,
        year,
        float(df_damaged_area["value"].iloc[0]),
        float(df_damaged_amount["value"].iloc[0]),
        float(df_damaged_ratio["value"].iloc[0])
        ]

    # 2017
    year = 2017
    path_csv = f"https://github.com/ShotaSasaki-HU/ASTRO-CAMP-2025/raw/refs/heads/main/data/%E6%B0%B4%E7%A8%B2%E3%81%AE%E8%A2%AB%E5%AE%B3%E9%9D%A2%E7%A9%8D%E5%8F%8A%E3%81%B2%E3%82%99%E8%A2%AB%E5%AE%B3%E9%87%8F%EF%BC%88%E5%85%A8%E5%9B%BD%E8%BE%B2%E6%A5%AD%E5%9C%B0%E5%9F%9F%E5%88%A5%E3%83%BB%E9%83%BD%E9%81%93%E5%BA%9C%E7%9C%8C%E5%88%A5%EF%BC%89/{year}_%E6%B0%B4%E7%A8%B2%E3%81%AE%E8%A2%AB%E5%AE%B3%E9%9D%A2%E7%A9%8D%E5%8F%8A%E3%81%B2%E3%82%99%E8%A2%AB%E5%AE%B3%E9%87%8F%EF%BC%88%E5%85%A8%E5%9B%BD%E8%BE%B2%E6%A5%AD%E5%9C%B0%E5%9F%9F%E5%88%A5%E3%83%BB%E9%83%BD%E9%81%93%E5%BA%9C%E7%9C%8C%E5%88%A5%EF%BC%89.csv"
    df = pd.read_csv(path_csv, encoding="shift-jis", header=0)

    target_col = [col for col in df.columns if "全国農業地域" in col][0] # 「全国農業地域」を含む列名を抽出
    df = df[df[target_col].str.contains(prefecture)]

    df = df[df["平成29水稲被害面積被害量"] == "虫害_ウンカ"]
    df_damaged_area = df[df["平成29水稲被害率被害量"] == "被害面積"]
    df_damaged_amount = df[df["平成29水稲被害率被害量"] == "被害量"]
    df_damaged_ratio = df[df["平成29水稲被害率被害量"] == "被害率_実数"]

    df_damage_ratio.loc[len(df_damage_ratio)] = [
        prefecture,
        year,
        float(df_damaged_area["value"].iloc[0]),
        float(df_damaged_amount["value"].iloc[0]),
        float(df_damaged_ratio["value"].iloc[0])
        ]

    # 2018
    year = 2018
    path_csv = f"https://github.com/ShotaSasaki-HU/ASTRO-CAMP-2025/raw/refs/heads/main/data/%E6%B0%B4%E7%A8%B2%E3%81%AE%E8%A2%AB%E5%AE%B3%E9%9D%A2%E7%A9%8D%E5%8F%8A%E3%81%B2%E3%82%99%E8%A2%AB%E5%AE%B3%E9%87%8F%EF%BC%88%E5%85%A8%E5%9B%BD%E8%BE%B2%E6%A5%AD%E5%9C%B0%E5%9F%9F%E5%88%A5%E3%83%BB%E9%83%BD%E9%81%93%E5%BA%9C%E7%9C%8C%E5%88%A5%EF%BC%89/{year}_%E6%B0%B4%E7%A8%B2%E3%81%AE%E8%A2%AB%E5%AE%B3%E9%9D%A2%E7%A9%8D%E5%8F%8A%E3%81%B2%E3%82%99%E8%A2%AB%E5%AE%B3%E9%87%8F%EF%BC%88%E5%85%A8%E5%9B%BD%E8%BE%B2%E6%A5%AD%E5%9C%B0%E5%9F%9F%E5%88%A5%E3%83%BB%E9%83%BD%E9%81%93%E5%BA%9C%E7%9C%8C%E5%88%A5%EF%BC%89.csv"
    df = pd.read_csv(path_csv, encoding="shift-jis", header=0)

    target_col = [col for col in df.columns if "全国農業地域" in col][0] # 「全国農業地域」を含む列名を抽出
    df = df[df[target_col].str.contains(prefecture)]

    df = df[df["(F002-30-1-012)被害量"] == "虫害_ウンカ"]
    df_damaged_area = df[df["(F002-30-1-011)被害項目"] == "被害面積"]
    df_damaged_amount = df[df["(F002-30-1-011)被害項目"] == "被害量"]
    df_damaged_ratio = df[df["(F002-30-1-011)被害項目"] == "被害率_実数"]

    df_damage_ratio.loc[len(df_damage_ratio)] = [
        prefecture,
        year,
        float(df_damaged_area["value"].iloc[0]),
        float(df_damaged_amount["value"].iloc[0]),
        float(df_damaged_ratio["value"].iloc[0])
        ]

    # 2019
    year = 2019
    path_csv = f"https://github.com/ShotaSasaki-HU/ASTRO-CAMP-2025/raw/refs/heads/main/data/%E6%B0%B4%E7%A8%B2%E3%81%AE%E8%A2%AB%E5%AE%B3%E9%9D%A2%E7%A9%8D%E5%8F%8A%E3%81%B2%E3%82%99%E8%A2%AB%E5%AE%B3%E9%87%8F%EF%BC%88%E5%85%A8%E5%9B%BD%E8%BE%B2%E6%A5%AD%E5%9C%B0%E5%9F%9F%E5%88%A5%E3%83%BB%E9%83%BD%E9%81%93%E5%BA%9C%E7%9C%8C%E5%88%A5%EF%BC%89/{year}_%E6%B0%B4%E7%A8%B2%E3%81%AE%E8%A2%AB%E5%AE%B3%E9%9D%A2%E7%A9%8D%E5%8F%8A%E3%81%B2%E3%82%99%E8%A2%AB%E5%AE%B3%E9%87%8F%EF%BC%88%E5%85%A8%E5%9B%BD%E8%BE%B2%E6%A5%AD%E5%9C%B0%E5%9F%9F%E5%88%A5%E3%83%BB%E9%83%BD%E9%81%93%E5%BA%9C%E7%9C%8C%E5%88%A5%EF%BC%89.csv"
    df = pd.read_csv(path_csv, encoding="shift-jis", header=0)

    target_col = [col for col in df.columns if "全国農業地域" in col][0] # 「全国農業地域」を含む列名を抽出
    df = df[df[target_col].str.contains(prefecture)]

    df = df[df["被害原因"] == "虫害_ウンカ"]
    df_damaged_area = df[df["表章項目"] == "被害面積"]
    df_damaged_amount = df[df["表章項目"] == "被害量"]
    df_damaged_ratio = df[df["表章項目"] == "被害率_実数"]

    df_damage_ratio.loc[len(df_damage_ratio)] = [
        prefecture,
        year,
        float(df_damaged_area["value"].iloc[0]),
        float(df_damaged_amount["value"].iloc[0]),
        float(df_damaged_ratio["value"].iloc[0])
        ]

    # 2020 ~ 2023
    for year in range(2020, 2023+1):
        path_csv = f"https://github.com/ShotaSasaki-HU/ASTRO-CAMP-2025/raw/refs/heads/main/data/%E6%B0%B4%E7%A8%B2%E3%81%AE%E8%A2%AB%E5%AE%B3%E9%9D%A2%E7%A9%8D%E5%8F%8A%E3%81%B2%E3%82%99%E8%A2%AB%E5%AE%B3%E9%87%8F%EF%BC%88%E5%85%A8%E5%9B%BD%E8%BE%B2%E6%A5%AD%E5%9C%B0%E5%9F%9F%E5%88%A5%E3%83%BB%E9%83%BD%E9%81%93%E5%BA%9C%E7%9C%8C%E5%88%A5%EF%BC%89/{year}_%E6%B0%B4%E7%A8%B2%E3%81%AE%E8%A2%AB%E5%AE%B3%E9%9D%A2%E7%A9%8D%E5%8F%8A%E3%81%B2%E3%82%99%E8%A2%AB%E5%AE%B3%E9%87%8F%EF%BC%88%E5%85%A8%E5%9B%BD%E8%BE%B2%E6%A5%AD%E5%9C%B0%E5%9F%9F%E5%88%A5%E3%83%BB%E9%83%BD%E9%81%93%E5%BA%9C%E7%9C%8C%E5%88%A5%EF%BC%89.csv"
        df = pd.read_csv(path_csv, encoding="shift-jis", header=0)

        target_col = [col for col in df.columns if "全国農業地域" in col][0] # 「全国農業地域」を含む列名を抽出
        df = df[df[target_col].str.contains(prefecture)]

        target_col = [col for col in df.columns if "被害面積及び被害量" in col][0]
        df_damaged_area = df[df[target_col] == "虫害_ウンカ_被害面積"]
        df_damaged_amount = df[df[target_col] == "虫害_ウンカ_被害量"]
        df_damaged_ratio = df[df[target_col] == "虫害_ウンカ_被害率_実数"]

        df_damage_ratio.loc[len(df_damage_ratio)] = [
            prefecture,
            year,
            float(df_damaged_area["value"].iloc[0]),
            float(df_damaged_amount["value"].iloc[0]),
            float(df_damaged_ratio["value"].iloc[0])
            ]

print(df_damage_ratio)


   都道府県  year  被害面積（ha）  被害量（t）  被害率（%）
0    福岡  2016    1280.0   182.0     0.1
1    福岡  2017    3010.0   495.0     0.3
2    福岡  2018    1120.0   121.0     0.1
3    福岡  2019    7060.0  2210.0     1.3
4    福岡  2020    9690.0  2890.0     1.7
5    福岡  2021     785.0   132.0     0.1
6    福岡  2022     686.0   203.0     0.1
7    福岡  2023     540.0    54.0     0.0
8    佐賀  2016    3140.0  1920.0     1.5
9    佐賀  2017    5550.0  2320.0     1.8
10   佐賀  2018     474.0    38.0     0.0
11   佐賀  2019    7630.0  6790.0     5.4
12   佐賀  2020    1470.0  1520.0     1.2
13   佐賀  2021     467.0   108.0     0.1
14   佐賀  2022     616.0   407.0     0.3
15   佐賀  2023      96.0    42.0     0.0
16   長崎  2016    1230.0   653.0     1.1
17   長崎  2017    2090.0   818.0     1.5
18   長崎  2018     539.0    33.0     0.1
19   長崎  2019    2980.0  1790.0     3.3
20   長崎  2020    2380.0  1540.0     2.9
21   長崎  2021     506.0    98.0     0.2
22   長崎  2022     606.0    91.0     0.2
23   長崎  2023     594.0   106.0     0.2


# NDVIのdf_ndvi

In [ ]:
trio_per_year = {
    2016: ["2016-03-28", "2016-04-30", "2016-06-16"],
    2017: ["2017-04-02", "2017-05-05", "2017-07-14"],
    2018: ["2018-03-28", "2018-04-25", "2018-06-16"],
    2019: ["2019-04-07", "2019-05-10", "2019-06-14"],
    2020: ["2020-04-16", "2020-05-19", "2020-06-15"],
    2021: ["2021-04-09", "2021-05-14", "2021-06-05"],
    2022: ["2022-04-11", "2022-05-16", "2022-06-25"],
    2023: ["2023-04-11", "2023-05-09", "2023-06-10"],
    2024: ["2024-03-29", "2024-05-08", "2024-05-28"],
    2025: ["2025-04-15", "2025-05-13", "2025-06-17"]
}

# 初期データフレーム
cols = ["year", "NDVI_4月", "NDVI_5月", "NDVI_6月"]
df_ndvi = pd.DataFrame(columns=cols)

for year in trio_per_year.keys():
    trio = trio_per_year[year]

    ndvi_means = []
    for date in trio:
        file_path = f"/content/drive/MyDrive/アストロキャンプ/試製/data/{date}.tif"

        # GeoTIFFを開く
        with rasterio.open(file_path) as src:
            # バンド数
            # print(f"バンド数: {src.count}")
            # サイズ
            # print(f"width: {src.width}, height: {src.height}")
            # 緯度経度の情報
            # print(src.crs)
            # print(src.descriptions)

            blue = src.read(4, masked=True)
            green = src.read(3, masked=True)
            red = src.read(2, masked=True)
            nir = src.read(1, masked=True)
            scl = src.read(5, masked=True)

        # NDVIを計算（浮動小数点にキャスト）
        red = red.astype(np.float32)
        nir = nir.astype(np.float32)
        ndvi = (nir - red) / (nir + red + 1e-6) # 0除算対策あり

        # sclの値が3, 8, 9のいずれかであるかを示すマスクを作成
        # np.isin()は、sclの各要素がリスト内の値と一致するかを調べる
        cloud_values = [3, 8, 9]
        cloud_mask = np.isin(scl, cloud_values)

        # clear_mask は cloud_mask の反対 (雲でない場所がTrue)
        clear_mask = ~cloud_mask

        # clear_maskを使ってNDVIから雲に該当するピクセルを除外（NaNにする）
        ndvi = np.where(clear_mask == 1, ndvi, np.nan)

        ndvi_means.append(np.nanmean(ndvi))

    df_ndvi.loc[len(df_ndvi)] = [
        year,
        ndvi_means[0],
        ndvi_means[1],
        ndvi_means[2]
    ]

print(df_ndvi)


# 地表面温度のdf_temp

In [ ]:
# df_temp = pd.read_csv("/content/drive/MyDrive/アストロキャンプ/試製/data/ground_surface_temperature.csv", index_col=0)
#
# # 文字列からdatetimeに変換
# df_temp["date"] = pd.to_datetime(df_temp["date"])
# # 年を文字列で取り出して新しい列を追加
# df_temp["year"] = df_temp["date"].dt.year.astype("int64")
#
# df_temp.drop(["date", "standard_deviation", "total_pixel", "valid_pixel"], axis=1, inplace=True)
#
# df_temp = df_temp.pivot(index="year", columns="month", values="average")
# df_temp = df_temp.reset_index()
# df_temp = df_temp.rename(columns={4: "地表temp_4月", 5: "地表temp_5月", 6: "地表temp_6月"})
#
# print(df_temp)

##############################

# 1. 定数を設定
# 分析対象領域 (Area of Interest)
region = ee.Geometry.Rectangle([116.14615, 28.91031, 116.16521, 28.92906])
# 対象とする年と月
start_year = 2016
end_year = 2024
months = [4, 5, 6]
# ウンカの発育下限温度（基準温度）。文献等を参考に設定するのが望ましい。
base_temperature = 12.0 # 佐賀のパラメータを参照

# 2. 結果を格納するための空のリストを準備
all_results = []

# 3. 年と月でループ処理
print(f"積算温度（基準温度: {base_temperature}°C）の計算を開始します...")
for year in range(start_year, end_year + 1):

    yearly_data = {'year': year}

    for month in months:
        # 月の開始日と終了日を定義
        start_date = f'{year}-{month:02d}-01'
        ee_start_date = ee.Date(start_date)
        ee_end_date = ee_start_date.advance(1, 'month')
        end_date = ee_end_date.format('YYYY-MM-dd').getInfo()

        # その月の日数を取得
        days_in_month = ee_end_date.difference(ee_start_date, 'day')

        # 指定した月のMODIS画像コレクションを取得
        collection = ee.ImageCollection('MODIS/061/MOD11A1').filterDate(start_date, end_date)

        # --- ここからが積算温度の計算部分 ---

        # 日々の積算温度を計算する関数を定義
        def calculate_degree_days(image):
            # LSTバンドを選択し、スケール変換と単位変換（ケルビン -> 摂氏）
            temp_celsius = image.select('LST_Day_1km').multiply(0.02).subtract(273.15)
            # (毎日の気温 - 基準温度) を計算。結果がマイナスなら0にする。
            degree_day = temp_celsius.subtract(base_temperature).max(0)
            return degree_day

        # コレクション内の各画像（各日）に対して上記の関数を適用
        daily_degree_days = collection.map(calculate_degree_days)

        # 月間の「日平均」積算温度を計算し、その月の日数を掛けて正規化する
        monthly_degree_days_image = daily_degree_days.mean().multiply(days_in_month).rename('Degree_Days')

        # AOI内の平均値を計算
        stats = monthly_degree_days_image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=region,
            scale=1000,
            maxPixels=1e9
        ).getInfo()

        degree_days_value = stats.get('Degree_Days')

        column_name = f'積算温度_{month}月'
        yearly_data[column_name] = degree_days_value

    all_results.append(yearly_data)
    print(f"{year}年の処理が完了しました。")

# 4. リストからpandasデータフレームを作成
df_degree_days = pd.DataFrame(all_results)

print("\n--- 完成したデータフレーム ---")
print(df_degree_days)


# dfへの統合

In [ ]:
# 正解データdf_damage_ratioをベースにする．
df = df_damage_ratio.copy() # 値渡し
df.drop(["被害面積（ha）", "被害量（t）"], axis=1, inplace=True)

# yearをキーにdf_ndviとdf_tempをマージ（左側優先）
df = df.merge(df_ndvi, on="year", how="left")
df = df.merge(df_degree_days, on="year", how="left")

df.dropna(inplace=True)
df.reset_index()
print("データ数:", len(df))
df


# 線形回帰

In [ ]:
# 説明変数と目的変数を選択
X = df[["NDVI_4月", "NDVI_5月", "NDVI_6月", "積算温度_4月", "積算温度_5月", "積算温度_6月"]].values
y = df["被害率（%）"].values # MODIS LST を目的変数にする例

# 学習/検証分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("係数:", model.coef_)
print("切片:", model.intercept_)
print("MSE:", mean_squared_error(y_test, y_pred))
print("R^2:", r2_score(y_test, y_pred))


# 散布図（実測 vs 予測）

In [ ]:
# 散布図（実測 vs 予測）
plt.scatter(y_test, y_pred, s=10)
plt.xlabel("被害率の実測値(%)")
plt.ylabel("被害率の予測値(%)")
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.legend(["実測値", "予測値"])
plt.show()


In [ ]:
# 横軸にする説明変数を選択
x_axis_var_index = 4 # 0 ~ 5
index_to_label = {
    0: "NDVI_4月",
    1: "NDVI_5月",
    2: "NDVI_6月",
    3: "積算温度_4月",
    4: "積算温度_5月",
    5: "積算温度_6月"
    }

# 実測値（y_test）を散布図に
plt.scatter(X_test[:, x_axis_var_index], y_test, label="Observed 被害率 (%)", alpha=0.7)

# 他の変数は平均値で固定して、回帰直線を作る
x_var_range = np.linspace(X_test[:, x_axis_var_index].min(),
                          X_test[:, x_axis_var_index].max(), 100)

X_fixed = np.zeros((100, X_test.shape[1]))
X_fixed[:, x_axis_var_index] = x_var_range

# 他の変数に平均値をセット
for i in range(X_test.shape[1]):
    if i != x_axis_var_index:
        X_fixed[:, i] = X_test[:, i].mean()

# モデルで予測
y_line = model.predict(X_fixed)

# 予測直線を描画
plt.plot(x_var_range, y_line, color="red", label="Regression line")

plt.xlabel(index_to_label[x_axis_var_index]) # 横軸名（選んだ変数に合わせて変更）
plt.ylabel("被害率（%）")
plt.legend()
plt.show()
